In [ ]:
import cv2 
import numpy as np 
from matplotlib import pyplot as plt
from pathlib import Path



In [ ]:
""" Au départ, nous pensions pouvoir charger directement chaque image avec un 
chemin relatif comme "Images/H&M.jpg", mais dans Jupyter le kernel ne se lance 
pas forcément dans le dossier du notebook. Pour éviter les erreurs de chemin, 
nous avons utilisé Path().resolve() pour parcourir les images du dossier. """ 

fichier = Path().resolve().parent / "Images" / "H&M.jpg"
img = cv2.imread(fichier, cv2.IMREAD_COLOR) 

cv2.imshow("Titre", img) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
